In [1]:
import json
import os
import random
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import scipy.sparse as sp

random.seed(42)
np.random.seed(42)

### Loading S-Values

We load the S-Values from the ChainRec model.

In [2]:
MAPPING_DIR = './mappings/'

s_values_df = pd.read_csv(MAPPING_DIR+"goodreads_s_values.csv")

In [3]:
cols_to_keep = ['user_number', 'item_number', 's1', 's2', 's3', 's4']

s_values_df = s_values_df[cols_to_keep]

In [4]:
s_values_df['user_number'] = s_values_df['user_number'].apply(lambda x: str(x))
s_values_df['item_number'] = s_values_df['item_number'].apply(lambda x: str(x))
s_values_df['user_item_id'] = s_values_df['user_number'] + "-" + s_values_df['item_number']

In [5]:
OUTPUT_DATA_DIR = "./output_data/"

train_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_training.csv")
val_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_validation.csv")
test_df_processed = pd.read_csv(OUTPUT_DATA_DIR+"text_processed_testing.csv")

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
print(len(train_df_processed))
print(len(val_df_processed))
print(len(test_df_processed))

257645
257645
257645


In [14]:
def load_mapping(mapping_file):
    """Loads the mapping from `mapping_file`.
    
    Parameters
    ----------
    mapping_file: str
        The name of the mapping file to import.
    
    Returns
    -------
    pd.DataFrame
        The DataFrame created from the mapping.
    
    """
    return pd.read_csv(os.path.join("mappings", "{}.csv".format(mapping_file)))

In [15]:
user_map = load_mapping("user_map")
book_map = load_mapping("book_map")

In [16]:
book_map['book_id'] = book_map['book_id'].apply(lambda x: str(x))

In [17]:
def create_user_item_id(data_df, u_map, i_map):
    data_df['book_id'] = data_df['book_id'].apply(lambda x: str(x))
    data_df = pd.merge(data_df, u_map, how="left", on=["user_id"])
    data_df = pd.merge(data_df, i_map, how="left", on=["book_id"])
    data_df['user_number'] = data_df['user_number'].apply(lambda x: str(x))
    data_df['book_number'] = data_df['book_number'].apply(lambda x: str(x))
    data_df['user_item_id'] = data_df['user_number'] + "-" + data_df['book_number']
    return data_df.drop(columns=['user_number', 'book_number'])

In [18]:
train_df = create_user_item_id(train_df_processed, user_map, book_map)
val_df = create_user_item_id(val_df_processed, user_map, book_map)
test_df = create_user_item_id(test_df_processed, user_map, book_map)

In [21]:
s_values_df.drop(columns=['user_number', 'item_number'], inplace=True)

In [26]:
train_df_s = pd.merge(train_df, s_values_df, how='left', on=['user_item_id'])
val_df_s = pd.merge(val_df, s_values_df, how='left', on=['user_item_id'])
test_df_s = pd.merge(test_df, s_values_df, how='left', on=['user_item_id'])

In [28]:
train_df_s.drop(columns=['user_item_id'], inplace=True)
val_df_s.drop(columns=['user_item_id'], inplace=True)
test_df_s.drop(columns=['user_item_id'], inplace=True)

In [29]:
train_df_s.to_csv(OUTPUT_DATA_DIR+"training_s_vals.csv", index=False)
val_df_s.to_csv(OUTPUT_DATA_DIR+"validation_s_vals.csv", index=False)
test_df_s.to_csv(OUTPUT_DATA_DIR+"testing_s_vals.csv", index=False)

In [68]:
columns_to_keep = ['text_reviews_count', 'is_ebook', 'average_rating', 'num_pages',
                   'ratings_count', 'is_translated', 'is_in_series',
                   'series_length', 'is_paperback', 'is_hardcover', 'is_audio', 'is_other_format',
                   'from_penguin', 'from_harpercollins', 'from_university_press', 'from_vintage',
                   'from_createspace', 'other_publisher', 'author_a', 'author_b', 'author_c',
                   'author_d', 'author_e', 'author_f', 'author_other', 's1', 's2', 's3', 's4']
X_train_reg = train_df_s[columns_to_keep]
X_val_reg = val_df_s[columns_to_keep]
X_test_reg = test_df_s[columns_to_keep]

In [69]:
def log_transform_columns(data_df, cols):
    """Applies a log transform to `cols` in `data_df`.

    Parameters
    ----------
    data_df: pd.DataFrame
        The DataFrame in which the columns will be transformed.
    cols: collection
        The columns in `data_df` to be log scaled.

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from `data_df` after log scaling
        the columns `cols`.

    """
    for col in cols:
        data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)
    return data_df

In [70]:
log_transform_cols = ['text_reviews_count', 'ratings_count']
X_train_reg = log_transform_columns(X_train_reg, log_transform_cols)
X_val_reg = log_transform_columns(X_val_reg, log_transform_cols)
X_test_reg = log_transform_columns(X_test_reg, log_transform_cols)

<ipython-input-69-49ee2668773d>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df[col] = data_df[col].apply(lambda x: np.log(x) if x > 0 else 0)


In [71]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler()

X_train_reg = min_max_scaler.fit_transform(X_train_reg)
X_val_reg = min_max_scaler.transform(X_val_reg)
X_test_reg = min_max_scaler.transform(X_test_reg)

In [34]:
book_df = train_df_s[['book_id', 'cleaned_text']]
book_df = book_df.drop_duplicates(subset=['book_id'])

book_df['cleaned_text'] = book_df['cleaned_text'].apply(lambda x: "" if pd.isnull(x) else x)

w2v = Word2Vec(list(book_df['cleaned_text']), size=200, window=10, min_count=1)

In [37]:
def create_book_vector(book_text, vec_length):
    """Creates a vector for the book given by `book_text`.

    The word vectors for each word in `book_text` are
    averaged to build a vector for the book.

    Parameters
    ----------
    book_text: str
        The book text for which the vector is generated.

    Returns
    -------
    vector
        A vector for the book.

    """
    text_vecs = [word for word in str(book_text) if word in w2v.wv.vocab]
    if len(text_vecs) > 0:
        return np.mean(w2v[text_vecs], axis=0)
    return np.zeros(vec_length)

In [38]:
train_df_s['book_vector'] = train_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))
val_df_s['book_vector'] = val_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))
test_df_s['book_vector'] = test_df_s['cleaned_text'].apply(lambda x: create_book_vector(x, 200))

<ipython-input-37-4533799a3578>:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  return np.mean(w2v[text_vecs], axis=0)


In [39]:
def create_book_vec_df(book_vecs, indices):
    """Creates a dataframe from `book_vecs`.

    Each numpy array in `book_vecs` is converted to a
    row in the resulting dataframe.

    Parameters
    ----------
    book_vecs: list
        A list of numpy arrays where each array corresponds
        to the book vector for a book.
    indicies: np.array
        A numpy array of indices for the DataFrame

    Returns
    -------
    pd.DataFrame
        The DataFrame obtained from converting `review_vecs`
        to a dataframe.

    """
    book_vec_df = pd.DataFrame(np.vstack(book_vecs))
    book_vec_df.columns = ["word" + str(col) for col in book_vec_df.columns]
    book_vec_df.index = indices
    return book_vec_df

In [40]:
train_wv = create_book_vec_df(train_df_s['book_vector'], train_df_s.index)
val_wv = create_book_vec_df(val_df_s['book_vector'], val_df_s.index)
test_wv = create_book_vec_df(test_df_s['book_vector'], test_df_s.index)

In [74]:
X_train_reg_df = pd.DataFrame(np.vstack(X_train_reg))
X_train_reg_df.index = train_df_s.index

X_val_reg_df = pd.DataFrame(np.vstack(X_val_reg))
X_val_reg_df.index = val_df_s.index

X_test_reg_df = pd.DataFrame(np.vstack(X_test_reg))
X_test_reg_df.index = test_df_s.index

X_train_wv_reg = sp.csr_matrix(pd.concat([train_wv, X_train_reg_df], axis=1))
X_val_wv_reg = sp.csr_matrix(pd.concat([val_wv, X_val_reg_df], axis=1))
X_test_wv_reg = sp.csr_matrix(pd.concat([test_wv, X_test_reg_df], axis=1))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

C_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]

for c in C_values:
    print("C = {}".format(c))
    print("------")
    reg_model = LogisticRegression(penalty='l1', C=c, max_iter=1000, solver='saga')
    reg_model.fit(X_train_wv_reg, train_df_s['recommended'])
    
    train_AUC = roc_auc_score(train_df_s['recommended'], reg_model.predict(X_train_wv_reg))
    val_AUC = roc_auc_score(val_df_s['recommended'], reg_model.predict(X_val_wv_reg))
    
    print("Training AUC: {}".format(train_AUC))
    print("Validation AUC: {}".format(val_AUC))
    print()

C = 0.1
------
Training AUC: 0.6771265588685002
Validation AUC: 0.6771265588685002

C = 0.3
------
